# Comprimiendo y descomprimiendo en *C*

En este notebook se va a analizar el rendimiento del algoritmo creado para poder comprimir los datos particionados y, de esta forma, poder descomprimir solo los datos necesarios.

Para ello se va a crear un conjunto de datos y se va a comprimir y descompirmir de la forma tradicional. Es decir, se va a descomprimir todo el conjunto de datos y, posteriormente, se seleccionará el subconjunto de datos deseado.

Por otra parte se va a particionar los datos y, luego, comprimirlos. A la hora de descomprimir, se calculará las particiones que contienen los datos deseados y sólo se descomprimirán estas.

## Paquetes necesarios

In [1]:
import transformData as td
import matplotlib.pyplot as plt
import tables as tb
import numpy as np
import time as t

## Definición del conjunto de datos

In [2]:
def file_reader(filename, dataset):
    with tb.open_file(filename) as f:
        child = f.root
        for element in dataset.split('/'):
            child = child.__getattr__(element)
        return child[:]

### Datos sin transformación

El conjunto de datos que se va a usar será de $4$ dimensiones con la forma $(744, 39, 30, 251)$. Por tanto habrá $744\cdot39\cdot30\cdot251 = 218.490.480$ elementos.

In [3]:
PATH = '/home/aleix/datasets/'

filename = 'WRF_India.h5'

dataset = 'U'

src = file_reader(PATH + filename, dataset)

SHAPE = src.shape

SIZE = np.prod(SHAPE)

ITEMSIZE = src.dtype.itemsize

DTYPE = src.dtype

### Datos transformados

El tamaño de las particiones tiene que ser más pequeño que el del conjunto de datos. En este caso, las particiones son de $32$ o $16$ elementos por cada dimensión lo que resulta que cada partición tendrá $32\cdot8\cdot8\cdot32=65.536$ elementos.

In [4]:
PART_SHAPE = [32, 8, 8, 32]

src_part = td.tData(src, PART_SHAPE, inverse=False)

TSHAPE = src_part.shape

## Compresión y descompresión de los datos

### Compresión

In [5]:
# Datos particionados

start = t.perf_counter()
dest_trans = td.compress(src_part, PART_SHAPE)
end = t.perf_counter()

ct_t = end - start

# Datos originales

start = t.perf_counter()
dest = td.compress(src, PART_SHAPE)
end = t.perf_counter()

c_t = end - start

### Descompresión

In [20]:
# Datos particionados

start = t.perf_counter()
res_trans = td.decompress_trans(dest_trans, SHAPE, TSHAPE, PART_SHAPE, DTYPE, a=500, d=200)
end = t.perf_counter()

dt_t = end - start

# Datos originales

start = t.perf_counter()
res = td.decompress(dest, SHAPE, DTYPE)[500, :, :, 200].reshape(res_trans.shape)
end = t.perf_counter()

d_t = end - start

## Resultados y conclusiones

In [21]:
try:
    np.testing.assert_array_equal(res_trans, res)
    print("OK. Las dos matrices son iguales.")
    print("El tiempo de decompresión de los datos particionados "
          "es {:.4f} veces más rápido que el de la decompresión "
          "normal.".format(d_t/dt_t))

except Exception:
    print("ERROR. Las dos matrices no coinciden.")

OK. Las dos matrices son iguales.
El tiempo de decompresión de los datos particionados es 155.0988 veces más rápido que el de la decompresión normal.
